In [1]:
import pandas as pd
import json

In [2]:
# Connect to database

from sqlalchemy import create_engine

URI="localhost"
PORT="5433"
DB = "jetaDb"
USER = "postgres"
PASSWORD = "00001234"
  
engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

/home/isaac/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# Variables from query

start = "271"
destination = "4096"
lineid = "44"
start_time = "1532018880"

# http://localhost:8000/main/journeytime?source=767&destination=1914&lineid=39A&time=1530008000
# http://localhost:8000/main/journeytime?source=719&destination=603&lineid=31B&time=1532018880
# Rain should be gotten from api or else table

rain = 0.5

In [4]:
# Transform time into variables required from model

import time
from datetime import datetime, timedelta
from pytz import timezone

# Get Irish timezone (utc + daylight saving time (DST))
irish_time = timezone('Europe/Dublin')
print("Irish Standard Time: ", irish_time)

# Get unixtime as datetime object
dt_time = datetime.fromtimestamp(int(start_time), irish_time)
print("Datetime: ", dt_time)

# Get day of week -> Mon: 0, Sun: 6
weekday = dt_time.weekday()
print("Weekday: ", weekday)

# Create list with desired weekday filled.
week_dummies = [0] * 7
week_dummies[weekday] = 1
del week_dummies[2] # Delete wednesday - not included in model due to dummy var trap
print("Week dummies: ", week_dummies)

# Get arrivaltime in seconds
date = dt_time.date()
date_unixtime = time.mktime(date.timetuple())
seconds_since_midnight = int(time.mktime((dt_time - timedelta(seconds = date_unixtime)).timetuple()))
print("Seconds since midnight (arrival time): ", seconds_since_midnight)


Irish Standard Time:  Europe/Dublin
Datetime:  2018-07-19 17:48:00+01:00
Weekday:  3
Week dummies:  [0, 0, 1, 0, 0, 0]
Seconds since midnight (arrival time):  64080


In [5]:
# Model inputs

model_inputs = [seconds_since_midnight, rain] + week_dummies
model_inputs

[64080, 0.5, 0, 0, 1, 0, 0, 0]

In [6]:
# Get stop lists associated with this lineid, start stop and end stop

sql = """

SELECT * 
FROM main_routes 
WHERE routeid IN (
    SELECT UNNEST(routes) 
    FROM main_lines 
    WHERE main_lines.lineid = '{0}'
) 
AND '{1}' = ANY(main_routes.stopids) 
AND '{2}' = ANY(main_routes.stopids) 
;

""".format(lineid, start, destination)

routes = pd.read_sql(sql, engine)

# Lines.objects.all
# Routes.objects.all(routeid )

2018-08-13 12:37:13,818 INFO sqlalchemy.engine.base.Engine select version()
2018-08-13 12:37:13,821 INFO sqlalchemy.engine.base.Engine {}
2018-08-13 12:37:13,833 INFO sqlalchemy.engine.base.Engine select current_schema()
2018-08-13 12:37:13,834 INFO sqlalchemy.engine.base.Engine {}
2018-08-13 12:37:13,846 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-13 12:37:13,847 INFO sqlalchemy.engine.base.Engine {}
2018-08-13 12:37:13,851 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-13 12:37:13,852 INFO sqlalchemy.engine.base.Engine {}
2018-08-13 12:37:13,857 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2018-08-13 12:37:13,858 INFO sqlalchemy.engine.base.Engine {}
2018-08-13 12:37:13,867 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [7]:
routes

,routeid,direction,stopids,lineid
0,44_38,1,"[208, 209, 210, 211, 212, 213, 214, 4432, 119,...",44
1,44_39,1,"[265, 271, 340, 350, 2809, 2810, 2811, 843, 10...",44
2,44_40,1,"[7571, 207, 208, 209, 210, 211, 212, 213, 214,...",44


In [8]:
if routes.shape[0] > 1:
    print("Error: multiple possible routes.")
    print(routes)

Error: multiple possible routes.
  routeid  direction                                            stopids lineid
0   44_38          1  [208, 209, 210, 211, 212, 213, 214, 4432, 119,...     44
1   44_39          1  [265, 271, 340, 350, 2809, 2810, 2811, 843, 10...     44
2   44_40          1  [7571, 207, 208, 209, 210, 211, 212, 213, 214,...     44


In [9]:
# Convert list of stopids to list

stop_list = routes['stopids'].tolist()[0]
print(stop_list)

[208, 209, 210, 211, 212, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 265, 271, 340, 350, 2809, 2810, 2811, 843, 1013, 1015, 2812, 2813, 2814, 852, 853, 854, 855, 2815, 2816, 2817, 2818, 2819, 2820, 2822, 2823, 5032, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 7375, 3466, 3467, 3469, 3470, 3471, 3472, 3473, 3474, 3475, 3476, 3477, 3478, 3280, 3282, 3479, 4086, 4087, 4088, 4089, 4090, 4091, 6097, 4093, 4094, 4096]


In [10]:
# Slice list by start and destination stop

journey_stops = stop_list[stop_list.index(int(start)):(stop_list.index(int(destination))+1)]
print(journey_stops)

[271, 340, 350, 2809, 2810, 2811, 843, 1013, 1015, 2812, 2813, 2814, 852, 853, 854, 855, 2815, 2816, 2817, 2818, 2819, 2820, 2822, 2823, 5032, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 7375, 3466, 3467, 3469, 3470, 3471, 3472, 3473, 3474, 3475, 3476, 3477, 3478, 3280, 3282, 3479, 4086, 4087, 4088, 4089, 4090, 4091, 6097, 4093, 4094, 4096]


In [11]:
# Remove duplicate values from list, while maintaining stop order.

from more_itertools import unique_everseen

journey_stops = list(unique_everseen(journey_stops))
print(journey_stops)

[271, 340, 350, 2809, 2810, 2811, 843, 1013, 1015, 2812, 2813, 2814, 852, 853, 854, 855, 2815, 2816, 2817, 2818, 2819, 2820, 2822, 2823, 5032, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 7375, 3466, 3467, 3469, 3470, 3471, 3472, 3473, 3474, 3475, 3476, 3477, 3478, 3280, 3282, 3479, 4086, 4087, 4088, 4089, 4090, 4091, 6097, 4093, 4094, 4096]


In [12]:
# Change each stopid into string
stringified = list(map(str, journey_stops))

# Zip ['0', .. 'n'] and ['1', .., 'n'] into list [('0', '1'), .., ('n - 1', 'n')]
# Join tuples to make ['1_2', .., 'n-1_n']
journey_segments = [ '_'.join(x) for x in zip(stringified[0:], stringified[1:])]
print(journey_segments)

['271_340', '340_350', '350_2809', '2809_2810', '2810_2811', '2811_843', '843_1013', '1013_1015', '1015_2812', '2812_2813', '2813_2814', '2814_852', '852_853', '853_854', '854_855', '855_2815', '2815_2816', '2816_2817', '2817_2818', '2818_2819', '2819_2820', '2820_2822', '2822_2823', '2823_5032', '5032_2824', '2824_2825', '2825_2826', '2826_2827', '2827_2828', '2828_2829', '2829_2830', '2830_2831', '2831_2832', '2832_2833', '2833_7375', '7375_3466', '3466_3467', '3467_3469', '3469_3470', '3470_3471', '3471_3472', '3472_3473', '3473_3474', '3474_3475', '3475_3476', '3476_3477', '3477_3478', '3478_3280', '3280_3282', '3282_3479', '3479_4086', '4086_4087', '4087_4088', '4088_4089', '4089_4090', '4090_4091', '4091_6097', '6097_4093', '4093_4094', '4094_4096']


In [13]:
journey_segments

['271_340',
 '340_350',
 '350_2809',
 '2809_2810',
 '2810_2811',
 '2811_843',
 '843_1013',
 '1013_1015',
 '1015_2812',
 '2812_2813',
 '2813_2814',
 '2814_852',
 '852_853',
 '853_854',
 '854_855',
 '855_2815',
 '2815_2816',
 '2816_2817',
 '2817_2818',
 '2818_2819',
 '2819_2820',
 '2820_2822',
 '2822_2823',
 '2823_5032',
 '5032_2824',
 '2824_2825',
 '2825_2826',
 '2826_2827',
 '2827_2828',
 '2828_2829',
 '2829_2830',
 '2830_2831',
 '2831_2832',
 '2832_2833',
 '2833_7375',
 '7375_3466',
 '3466_3467',
 '3467_3469',
 '3469_3470',
 '3470_3471',
 '3471_3472',
 '3472_3473',
 '3473_3474',
 '3474_3475',
 '3475_3476',
 '3476_3477',
 '3477_3478',
 '3478_3280',
 '3280_3282',
 '3282_3479',
 '3479_4086',
 '4086_4087',
 '4087_4088',
 '4088_4089',
 '4089_4090',
 '4090_4091',
 '4091_6097',
 '6097_4093',
 '4093_4094',
 '4094_4096']

In [14]:
# Select coefficient rows with these segment ids

sql2 = """

SELECT *
FROM main_coefficients
WHERE segment = ANY(ARRAY{0})

""".format(journey_segments)

coefficients = pd.read_sql(sql2, engine)

2018-08-13 12:37:21,009 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-08-13 12:37:21,011 INFO sqlalchemy.engine.base.Engine {'name': "\n\nSELECT *\nFROM main_coefficients\nWHERE segment = ANY(ARRAY['271_340', '340_350', '350_2809', '2809_2810', '2810_2811', '2811_843', '843_1013', ' ... (535 characters truncated) ... , '3282_3479', '3479_4086', '4086_4087', '4087_4088', '4088_4089', '4089_4090', '4090_4091', '4091_6097', '6097_4093', '4093_4094', '4094_4096'])\n\n"}
2018-08-13 12:37:21,081 INFO sqlalchemy.engine.base.Engine 

SELECT *
FROM main_coefficients
WHERE segment = ANY(ARRAY['271_340', '340_350', '350_2809', '2809_2810', '2810_2811', '2811_843', '843_1013', '1013_1015', '1015_2812', '2812_2813', '2813_2814', '2814_852', '852_853', '853_854', '854_855', '855_2815', '2815_2816', '2816_2817', '2817_2818', '2818_2819', '2819_2820', '2820_2822', '2822

In [15]:
coefficients

,segment,intercept,arrivaltime,rain,fri,mon,sat,sun,thu,tue
0,854_855,72.635642,-2.385477e-04,1.989291,0.696891,-0.920012,-13.891177,-14.729322,-0.782394,2.322896
1,2809_2810,120.907632,-3.157911e-04,-1.135140,-2.032439,-5.365322,-17.815800,-19.656937,-3.927563,-4.102119
2,3475_3476,43.140046,1.425080e-04,0.346994,2.539688,-2.171400,0.711908,-2.726691,-0.177923,0.362943
3,4086_4087,45.032051,-5.802301e-05,0.136039,0.601447,-0.352136,0.057754,-0.737383,0.533472,0.546205
4,4093_4094,68.788258,-1.232187e-04,-0.388623,0.013348,0.421483,-0.123807,-1.390882,1.856005,1.021528
5,3280_3282,34.732143,-3.189837e-05,-0.798636,3.096775,0.427299,0.109030,3.114242,1.776163,1.223088
6,3466_3467,51.864135,-8.287313e-05,-0.127626,1.247289,-2.347294,-0.209045,0.388275,-0.951660,-0.435490
7,2825_2826,85.999026,-2.817501e-04,0.887450,10.153806,2.565345,2.974227,-6.281726,4.645551,7.479638
8,2826_2827,77.711744,1.366822e-04,3.818211,5.523855,-11.083705,1.472919,-6.176899,-5.119667,-4.595464
9,3467_3469,142.986843,-4.329765e-04,-2.731432,-1.137977,-4.511173,-6.972325,-9.459415,3.179512,-2.796414


In [16]:
# Sort values by journey_segment segmentid

coefficients['segment'] = coefficients['segment'].astype("category")
coefficients['segment'].cat.set_categories(journey_segments, inplace=True)
coefficients = coefficients.sort_values(["segment"])
coefficients

,segment,intercept,arrivaltime,rain,fri,mon,sat,sun,thu,tue
39,271_340,161.534817,-3.563930e-05,-0.150783,0.070640,-1.950645,-0.669941,-2.227739,-0.468549,-0.969671
28,340_350,141.079364,-3.125280e-04,1.206848,-3.461752,-9.653060,-11.287650,-11.749047,-4.555987,-4.414771
29,350_2809,158.707592,-6.104254e-04,3.370105,4.360411,-2.686285,-18.706899,-18.071784,0.987693,3.155045
1,2809_2810,120.907632,-3.157911e-04,-1.135140,-2.032439,-5.365322,-17.815800,-19.656937,-3.927563,-4.102119
37,2810_2811,60.308766,-2.280988e-04,5.393459,-6.208182,-11.561753,-16.357301,-16.374387,-6.570648,-7.023982
11,2811_843,173.091101,-3.569364e-04,0.608130,-2.401580,-8.024743,-13.418403,-14.193066,-6.616014,-7.498881
55,843_1013,97.610825,-1.165566e-04,3.125801,3.785965,-5.851937,-7.394060,-14.858989,3.527198,0.707090
23,1013_1015,120.646297,-5.012656e-04,1.648140,-3.386100,-6.896687,-24.488820,-23.172373,-2.671421,-2.061198
19,1015_2812,94.849995,-2.330385e-04,-0.235809,-1.222745,-5.644576,-20.946330,-24.982560,0.059547,-0.956513
52,2812_2813,111.946931,-2.112071e-04,-2.555637,2.975812,-7.085104,-12.447193,-18.222132,-6.010086,1.210663


In [17]:
# Rearrange columns and set segment id as index

coefficients = coefficients[["segment", "intercept", "arrivaltime", "rain", "mon", "tue", "thu", "fri", "sat", "sun"]]
coefficients = coefficients.set_index('segment')
coefficients.head(5)

,intercept,arrivaltime,rain,mon,tue,thu,fri,sat,sun
segment,,,,,,,,,
271_340,161.534817,-0.000036,-0.150783,-1.950645,-0.969671,-0.468549,0.070640,-0.669941,-2.227739
340_350,141.079364,-0.000313,1.206848,-9.653060,-4.414771,-4.555987,-3.461752,-11.287650,-11.749047
350_2809,158.707592,-0.000610,3.370105,-2.686285,3.155045,0.987693,4.360411,-18.706899,-18.071784
2809_2810,120.907632,-0.000316,-1.135140,-5.365322,-4.102119,-3.927563,-2.032439,-17.815800,-19.656937
2810_2811,60.308766,-0.000228,5.393459,-11.561753,-7.023982,-6.570648,-6.208182,-16.357301,-16.374387


In [18]:
arrivaltime = model_inputs[0]
totaltraveltime = 0
segment_times = []

for i, rows in coefficients.iterrows():
    traveltime = (rows['intercept']
                  +(rows['arrivaltime']*arrivaltime)
                  +(rows['rain']*model_inputs[1])
                  +(rows['fri']*model_inputs[2])
                  +(rows['mon']*model_inputs[3])
                  +(rows['sat']*model_inputs[4])
                  +(rows['sun']*model_inputs[5])
                  +(rows['thu']*model_inputs[6])
                  +(rows['fri']*model_inputs[7]))
    
    segment_times.append((i, round(traveltime)))
    totaltraveltime += traveltime
    arrivaltime = model_inputs[0] + totaltraveltime # initial start time + sum of previous segment times
        
print('Arrival Time:', int(arrivaltime))
print('Total Travel Time:', int(totaltraveltime))
#print('Segment Times:', segment_times)
segment_times

Arrival Time: 67510
Total Travel Time: 3430


[('271_340', 159.0),
 ('340_350', 110.0),
 ('350_2809', 102.0),
 ('2809_2810', 82.0),
 ('2810_2811', 32.0),
 ('2811_843', 137.0),
 ('843_1013', 84.0),
 ('1013_1015', 65.0),
 ('1015_2812', 59.0),
 ('2812_2813', 85.0),
 ('2813_2814', 69.0),
 ('2814_852', 92.0),
 ('852_853', 78.0),
 ('853_854', 30.0),
 ('854_855', 44.0),
 ('855_2815', 77.0),
 ('2815_2816', 32.0),
 ('2816_2817', 57.0),
 ('2817_2818', 52.0),
 ('2818_2819', 34.0),
 ('2819_2820', 32.0),
 ('2820_2822', 30.0),
 ('2822_2823', 43.0),
 ('2823_5032', 28.0),
 ('5032_2824', 43.0),
 ('2824_2825', 47.0),
 ('2825_2826', 71.0),
 ('2826_2827', 90.0),
 ('2827_2828', 80.0),
 ('2828_2829', 91.0),
 ('2829_2830', 42.0),
 ('2830_2831', 36.0),
 ('2831_2832', 38.0),
 ('2832_2833', 50.0),
 ('2833_7375', 32.0),
 ('7375_3466', 69.0),
 ('3466_3467', 46.0),
 ('3467_3469', 106.0),
 ('3469_3470', 51.0),
 ('3470_3471', 37.0),
 ('3471_3472', 37.0),
 ('3472_3473', 56.0),
 ('3473_3474', 47.0),
 ('3474_3475', 63.0),
 ('3475_3476', 54.0),
 ('3476_3477', 33.0)

In [19]:
total = 0
for i in segment_times:
    total += i[1]
    
print(total)

3432.0


In [20]:
# Construct json

json_dict = {}
json_dict['arrivaltime'] = round(arrivaltime)
json_dict['totaltraveltime'] = round(totaltraveltime)
json_dict['segment_times'] = {i[0]:i[1] for i in segment_times}

# for i in segment_times:
#     json_dict['segment_times'][i[0]] = i[1]

In [21]:
json.dumps(json_dict)

'{"arrivaltime": 67511.0, "totaltraveltime": 3431.0, "segment_times": {"271_340": 159.0, "340_350": 110.0, "350_2809": 102.0, "2809_2810": 82.0, "2810_2811": 32.0, "2811_843": 137.0, "843_1013": 84.0, "1013_1015": 65.0, "1015_2812": 59.0, "2812_2813": 85.0, "2813_2814": 69.0, "2814_852": 92.0, "852_853": 78.0, "853_854": 30.0, "854_855": 44.0, "855_2815": 77.0, "2815_2816": 32.0, "2816_2817": 57.0, "2817_2818": 52.0, "2818_2819": 34.0, "2819_2820": 32.0, "2820_2822": 30.0, "2822_2823": 43.0, "2823_5032": 28.0, "5032_2824": 43.0, "2824_2825": 47.0, "2825_2826": 71.0, "2826_2827": 90.0, "2827_2828": 80.0, "2828_2829": 91.0, "2829_2830": 42.0, "2830_2831": 36.0, "2831_2832": 38.0, "2832_2833": 50.0, "2833_7375": 32.0, "7375_3466": 69.0, "3466_3467": 46.0, "3467_3469": 106.0, "3469_3470": 51.0, "3470_3471": 37.0, "3471_3472": 37.0, "3472_3473": 56.0, "3473_3474": 47.0, "3474_3475": 63.0, "3475_3476": 54.0, "3476_3477": 33.0, "3477_3478": 57.0, "3478_3280": 24.0, "3280_3282": 32.0, "3282_34